In [1]:
import os
import datetime
import tensorflow as tf
import tensorflow_models as tfm
import tensorflow_probability as tfp
from preprocess_data import load_data
import numpy as np
import logging
from typing import Tuple, Optional

2024-08-06 10:22:44.071071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 10:22:45.662264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import keras_nlp

Using TensorFlow backend


In [3]:
train_dataset = tf.data.experimental.load(
    "/home/annalena/PedestrianTrajectoryPrediction/tests/train_dataset_scaled"
    )

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


2024-08-06 10:23:23.598865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 10:23:23.813376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 10:23:23.814041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 10:23:23.817459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06 10:23:23.818300: I tensorflow/compile

In [4]:
len(train_dataset)

6608

In [5]:
class PreprocessLayer(tf.keras.layers.Layer):
    """ Applies the masking to the sequence
    """

    def __init__(self):
      super().__init__()

    def calc_hidden_mask(self, batch_size=32, sequence_length=15):
        # create mask array, False = needs to be predicted
        mask_arrays = []
        #print("batch_size:", batch_size)
        #print("sequence_length:", sequence_length)
        for i in range(batch_size):
          mask_arr = [True] * 6 + [False] * (sequence_length-6)
          # hide 0-2 in between steps (for lazyness whole datapoint)
          hidden_nr = np.random.randint(3)
          hidden_idx = np.random.choice(range(6),hidden_nr, replace=False)
          for v in hidden_idx:
              mask_arr[v] = False
          mask_arrays.append(mask_arr)
        #print("mask aaray:", np.asarray(mask_arrays).shape)
        return np.asarray(mask_arrays)

    def call(self,
           raw_input_batch: Tuple[tf.Tensor, tf.Tensor],
           is_hidden: Optional[tf.Tensor] = None) -> Tuple[Tuple[tf.Tensor, tf.Tensor], tf.Tensor]:
        input_batch = raw_input_batch
  

        batch_size = tf.shape(input_batch[0])[0]
        sequence_length = tf.shape(input_batch[0])[1]
        feature_size1 = tf.shape(input_batch[0])[2]
        feature_size2 = tf.shape(input_batch[1])[2]

        mask = self.calc_hidden_mask() #tf.convert_to_tensor

        mask_tensor = tf.constant(mask, dtype=tf.bool)

        # Expand dimensions of mask to match the input tensor
        #expanded_mask = tf.expand_dims(mask_tensor, axis=0)  # Add batch dimension
        expanded_mask = tf.expand_dims(mask_tensor, axis=-1)  # Add feature dimension

        # Broadcast mask to match input tensor shape
        broadcasted_mask_pos = tf.broadcast_to(expanded_mask, (batch_size, sequence_length, feature_size1))
        broadcasted_mask_pose = tf.broadcast_to(expanded_mask, (batch_size, sequence_length, feature_size2))

        #batch_mask = tf.broadcast_to(expanded_mask, (batch_size, sequence_length))

        # Apply mask
        masked_input_pos = tf.where(broadcasted_mask_pos, input_batch[0], tf.zeros_like(input_batch[0]))
        masked_input_pose = tf.where(broadcasted_mask_pose, input_batch[1], tf.zeros_like(input_batch[1]))
        targets = tf.where(tf.math.logical_not(broadcasted_mask_pos), input_batch[0], tf.zeros_like(input_batch[0]))

      # scale
        #scale_factor = 100.0
        #masked_input_pos = tf.math.scalar_mul(scale_factor, masked_input_pos)
        #masked_input_pose = tf.math.scalar_mul(scale_factor, masked_input_pose) 

        return (masked_input_pos, masked_input_pose), mask, targets

In [5]:
class PreprocessLayer_2(tf.keras.layers.Layer):
    """ Applies the masking to the sequence
    """

    def __init__(self, num_steps=15, num_history_steps=5):
        super().__init__()
        self._num_steps = num_steps
        self._num_history_steps = num_history_steps

    # from BPIsHiddenGenerator
    def calc_hidden_mask(self, batch_size=32, sequence_length=15):
        """Returns the is_hidden tensor for behavior prediction.

        Always returns 0 (not hidden) for history/current steps and 1 (hidden)
        for future steps.

        Args:
        num_agents: Number of agents in the scene.
        train_progress: A float between 0 to 1 representing the overall progress
            of training. This float can be current_step / total_training_steps. This
            float can be used for training w/ an annealing schedule.

        Returns:
        is_hidden: The is_hidden tensor for behavior prediction.
        """
        # [1, a, t, 1].
        is_hidden = np.ones([1, self._num_steps, 1],
                            dtype=bool)
        is_hidden[:, :self._num_history_steps + 1, :] = False
        return is_hidden
    
    def _set_elems_to_value(self, target, should_set, new_val):
        target = tf.where(should_set, tf.cast(new_val, target.dtype), target)
        return target

    def call(self,
           raw_input_batch: Tuple[tf.Tensor, tf.Tensor]) -> Tuple[Tuple[tf.Tensor, tf.Tensor], tf.Tensor]:
        input_batch = raw_input_batch
  
        is_hidden = self.calc_hidden_mask() #tf.convert_to_tensor
        print("is_hidden", is_hidden.shape)

        input_batch_new = []

        for feature_batch in input_batch:
            input_batch_new.append(tf.where(
            tf.math.is_nan(feature_batch),
            tf.zeros_like(feature_batch),
            feature_batch))
        
        print("input_batch_new 0", input_batch_new[0].shape)
        print("input_batch_new 1", input_batch_new[1].shape)
        
        t = input_batch_new[0]
        if t.dtype.is_floating:
            has_data =  tf.math.logical_not(
                tf.reduce_any(tf.math.is_nan(t), axis=-1, keepdims=True))
        else:
            has_data =  tf.math.logical_not(
                tf.reduce_any(t == t.dtype.min, axis=-1, keepdims=True))
            
        print("has_data", has_data.shape)
        
        has_historic_data = tf.reduce_any( has_data[..., :self._num_history_steps + 1, :], axis=-2, keepdims=True)
        should_predict = tf.logical_and(is_hidden,tf.logical_and(has_data, has_historic_data))

        print("has_historic_data", has_historic_data.shape)
        print("should_predict", should_predict.shape)
        
        feature_is_padded = tf.logical_not(has_data)
        masked_input_pos = self._set_elems_to_value( input_batch_new[0], tf.logical_or(feature_is_padded,is_hidden), 0.)
        masked_input_pose = self._set_elems_to_value( input_batch_new[1], tf.logical_or(feature_is_padded,is_hidden), 0.)

        print("masked_input_pos", masked_input_pos.shape)
        print("masked_input_pose", masked_input_pose.shape)
        targets = input_batch_new[0]

        output_dict={
            "masked_inputs": (masked_input_pos, masked_input_pose),
            "has_data": has_data, 
            "is_hidden": is_hidden, 
            "targets": targets,
            "has_historic_data": has_historic_data
        }

        return output_dict

In [67]:
class Test(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.preprocess_layer = PreprocessLayer_2() 

    def call(self, input_batch, training = False):
        (input_1, input_2) = input_batch
        output_dict = self.preprocess_layer((input_1, input_2)) # output shape (batch_size, 15, 3)

        return output_dict
    
strategy = tf.distribute.OneDeviceStrategy('gpu')
with strategy.scope():
    model = Test()
    model.compile(loss='msle', optimizer='rmsprop')

for (batch_x1, batch_x2) in train_dataset.take(1):
    input_batch = (batch_x1, batch_x2)
    output = model(input_batch, training=False)
    break

is_hidden (1, 15, 1)
input_batch_new 0 (32, 15, 3)
input_batch_new 1 (32, 15, 51)
has_data (32, 15, 1)
has_historic_data (32, 1, 1)
should_predict (32, 15, 1)
masked_input_pos (32, 15, 3)
masked_input_pose (32, 15, 51)


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 15207561690904214291


In [18]:
output["is_hidden"][0]

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

In [170]:
""" Adapted Sinusoidal Embedding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/embedding.py    """
class SinusoidalEmbeddingLayer(tf.keras.layers.Layer):
  """Sinusoidal Postional Embedding for xyz and time."""

  def __init__(self, min_freq=4, max_freq=256, hidden_size=256):
    super().__init__()
    self.min_freq = float(min_freq)
    self.max_freq = float(max_freq)
    self.hidden_size = hidden_size
    if hidden_size % 2 != 0:
      raise ValueError('hidden_size ({hidden_size}) must be divisible by 2.')
    self.num_freqs_int32 = hidden_size // 2
    self.num_freqs = tf.cast(self.num_freqs_int32, dtype=tf.float32)

  def build(self, input_shape):
    log_freq_increment = (
        tf.math.log(float(self.max_freq) / float(self.min_freq)) /
        tf.maximum(1.0, self.num_freqs - 1))
    # [num_freqs]
    self.inv_freqs = self.min_freq * tf.exp(
        tf.range(self.num_freqs, dtype=tf.float32) * -log_freq_increment)

  def call(self, input_tensor):
    
    # [batch_size, sequence_length, feature_size, num_freqs]
    input_tensor = tf.expand_dims(input_tensor, axis=-1)
    input_tensor = tf.repeat(input_tensor, self.num_freqs_int32, axis=-1)

    # [batch_size, sequence_length, feature_size, hidden_size]
    embedded = tf.concat([
        tf.sin(input_tensor * self.inv_freqs),
        tf.cos(input_tensor * self.inv_freqs)
    ], axis=-1)
    return embedded

In [6]:
""" Adapted Agent Position Encoding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/agent_feature_encoder.py    """
class AgentPositionEncoder(tf.keras.layers.Layer):
  """Encodes agents spatial positions."""

  def __init__(self, output_shape, embedding_size):
    
    super().__init__()

    #self.embedding_layer = SinusoidalEmbeddingLayer(
       # hidden_size=embedding_size) # output_shape (batch_sie, sequence_length, feature size, hidden_size)
    self.embedding_layer = keras_nlp.layers.SinePositionEncoding(max_wavelength=10000)
    #self.layer_norm = tf.keras.layers.LayerNormalization(axis=-1)  
    self.mlp = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        
        activation=None)

  """def call(self, input_batch):
    normalized_input = input_batch[0] #self.layer_norm(input_batch[0])
    embedded_input = self.embedding_layer(normalized_input)
    return self.mlp(embedded_input)"""
  def call(self, input_dict):
    is_hidden = input_dict["is_hidden"]
    has_data = input_dict["has_data"]
    input_batch = input_dict["masked_inputs"]
    not_is_hidden = tf.logical_not(is_hidden)
    mask = tf.logical_and(has_data, not_is_hidden)
    mask = tf.repeat(mask, tf.shape(input_batch[0])[-1], axis=-1)
    return self.mlp(self.embedding_layer(input_batch[0]))[..., tf.newaxis, :], mask
class AgentTemporalEncoder(tf.keras.layers.Layer):
  """Encodes agents temporal positions."""

  def __init__(self,output_shape, embedding_size, num_steps):
    super().__init__()
    #self.embedding_layer = SinusoidalEmbeddingLayer(
        #max_freq=num_steps,
        #hidden_size=embedding_size)
    self.embedding_layer = keras_nlp.layers.SinePositionEncoding(max_wavelength=10000)

    self.mlp = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        activation=None)

  def _get_temporal_embedding(self, input_batch):
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_batch[0])[0]
    num_steps = tf.shape(input_batch[0])[1]

    t = tf.range(0, num_steps, dtype=tf.float32)
    t = t[tf.newaxis, :]
    t = tf.tile(t, [b, 1])
    return self.embedding_layer(t[..., tf.newaxis])

  def call(self, input_dict):
    has_data = input_dict["has_data"]
    input_batch = input_dict["masked_inputs"]
    return (self.mlp(self._get_temporal_embedding(input_batch))[..., tf.newaxis, :],
            tf.ones_like(has_data))
  

  """ Adapted Agent Keypoint Encoding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/agent_feature_encoder.py    """
class AgentKeypointsEncoder(tf.keras.layers.Layer):
  """Encodes the agent's keypoints."""

  def __init__(self, output_shape, embedding_size):
    super().__init__()

    self.mlp1 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=output_shape,
        bias_axes='h',
        activation=tf.nn.relu)

  """def call(self, input_batch, training=None):

    keypoints = input_batch[1]

    out = self.mlp1(keypoints)

    return out"""
  def call(self, input_dict, training=None):
    is_hidden = input_dict["is_hidden"]
    has_data = input_dict["has_data"]
    input_batch = input_dict["masked_inputs"]
    not_is_hidden = tf.logical_not(is_hidden)
    mask = tf.logical_and(has_data, not_is_hidden)

    keypoints = input_batch[1]

    out = self.mlp1(keypoints)[..., tf.newaxis, :]

    return out, mask


In [58]:
class FeatureConcatAgentEncoderLayer(tf.keras.layers.Layer):
  """Independently encodes features and attends to them.

  Agent features are cross-attended with a learned query or hidden_vecs instead
  of MLP.
  """

  def __init__(self, input_length, batch_size=32, hidden_size=128, num_heads=4, ln_eps=1e-6, transformer_ff_dim=128, drop_prob=0.1):
    super().__init__()

    # Cross Attention and learned query.
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None,
    )
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)

    self.agent_feature_embedding_layers = []
    # Position Feature [batch, sequence_len, feature_size, hidden_size]
    self.agent_feature_embedding_layers.append(
        AgentPositionEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))
    # Feature Embedding - keypoints [batch, sequence_len, hidden_size]
    self.agent_feature_embedding_layers.append(
        AgentKeypointsEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))

    # Temporal Embedding [batch, sequence_len, 1, hidden_size]
    self.agent_feature_embedding_layers.append(
        AgentTemporalEncoder(output_shape=hidden_size-8, embedding_size=hidden_size, num_steps=input_length))


  def call(self, input_dict, training = None):
    is_hidden = input_dict["is_hidden"]
    has_data = input_dict["has_data"]
    input_batch = input_dict["masked_inputs"]
    layer_embeddings = []
    for layer in self.agent_feature_embedding_layers:
      layer_embedding, _ = layer(input_dict, training=training)
      print(layer_embedding.shape)
      layer_embedding = layer_embedding[...,0,:]
      print(layer_embedding.shape)
      layer_embeddings.append(layer_embedding)

    embedding = tf.concat(layer_embeddings, axis=-1)
    print("concat embedding", embedding.shape)

    out = self.ff_layer2(embedding)

    return out

In [38]:
""" Adapted Feature Encoding Layer from source: https://github.com/google-research/human-scene-transformer/blob/main/human_scene_transformer/model/agent_encoder.py    """
class FeatureAttnAgentEncoderLearnedLayer(tf.keras.layers.Layer):
  """Independently encodes features and attends to them.

  Agent features are cross-attended with a learned query or hidden_vecs instead
  of MLP.
  """

  def __init__(self, input_length=15, batch_size=32, hidden_size=128, num_heads=4, ln_eps=1e-6, transformer_ff_dim=128, drop_prob=0.1):
    super(FeatureAttnAgentEncoderLearnedLayer, self).__init__()

    self.batch_size=batch_size
    self.input_length = input_length
    self.num_heads=num_heads

    # Cross Attention and learned query.
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        #key_dim=360//num_heads,  # "large" to prevent a bottleneck
        #value_dim=360//num_heads)
        key_dim= hidden_size,
        attention_axes=1)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f',
        output_shape=transformer_ff_dim,
        bias_axes='f',
        activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.agent_feature_embedding_layers = []
    # Position Feature
    self.agent_feature_embedding_layers.append(
        AgentPositionEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))
    # Feature Embedding - keypoints
    self.agent_feature_embedding_layers.append(
        AgentKeypointsEncoder(output_shape=hidden_size-8, embedding_size=hidden_size))

    # Temporal Embedding
    self.agent_feature_embedding_layers.append(
        AgentTemporalEncoder(output_shape=hidden_size-8, embedding_size=hidden_size, num_steps=input_length))

    # [1, 1, h]
    self.learned_query_vec = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1., maxval=1.)(shape=[1, 1, 1, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def _build_learned_query(self, input_dict):
    """Converts self.learned_query_vec into a learned query vector."""
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_dict["masked_inputs"][0])[0]
    num_steps = tf.shape(input_dict["masked_inputs"][0])[1]

    # [b, num_steps, h]s
    return tf.tile(self.learned_query_vec, [b, num_steps, 1,1])

  def call(self, input_dict,training = None):
    
    layer_embeddings = []
    layer_masks = []
    for layer in self.agent_feature_embedding_layers:
      layer_embedding, layer_mask = layer(input_dict, training=training)
      print("layer_mask", layer_mask.shape)
      layer_embeddings.append(layer_embedding)
      layer_masks.append(layer_mask)
    embedding = tf.concat(layer_embeddings, axis=2)
    print("embedding:", embedding.shape)

    b = tf.shape(embedding)[0]
    t = tf.shape(embedding)[1]
    n = tf.shape(embedding)[2]

    # [1, 1, 1, N, 8]
    one_hot = tf.one_hot(tf.range(0, n), 8)[None, None]
    print("one_hot:", one_hot.shape)
    # [b, a, t, N, 8]
    one_hot_id = tf.tile(one_hot, (b, t, 1, 1))
    print("one_hot_id:", one_hot_id.shape)

    embedding = tf.concat([embedding, one_hot_id], axis=-1)
    print("embedding:", embedding.shape)

    attention_mask = tf.concat(layer_masks, axis=-1)
    print("attention_mask:", attention_mask.shape)

    # [b, a, t, num_heads, 1, num_features] <- broadcasted
    # Newaxis for num_heads, num_features
    attention_mask = attention_mask[..., tf.newaxis, tf.newaxis,:]
    print("attention_mask:", attention_mask.shape)
    attention_mask = tf.reshape(attention_mask, [b, 1, t, n])
    print("attention_mask:", attention_mask.shape)

    learned_query = self._build_learned_query(input_dict)
    print("learned_query:", learned_query.shape)

    # Attention along axis 3
    attn_out, attn_score = self.attn_layer(
        query=learned_query,
        key=embedding,
        value=embedding,
        attention_mask=attention_mask,
        return_attention_scores=True)
    print("attn_out shape: ", attn_out.shape)
    # [b, t, h]
    attn_out = attn_out[..., 0, :]
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    return out, attn_score

In [12]:
class AgentSelfAlignmentLayer(tf.keras.layers.Layer):
  """Enables agent to become aware of its temporal identity.

  Agent features are cross-attended with a learned query in temporal dimension.
  """

  def __init__(self,
               num_heads=8,
               hidden_size=128,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_heads=num_heads
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=ff_dim // num_heads,
        #value_dim = ff_dim // num_heads,
        attention_axes=1)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    # [1, 1, h]
    self.learned_query_vec = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1., maxval=1.)(shape=[1, 1, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def build_learned_query(self, input_dict):
    """Converts self.learned_query_vec into a learned query vector."""
    # This weird thing is for exporting and loading keras model...
    b = tf.shape(input_dict['hidden_vecs'])[0]
    t = tf.shape(input_dict['hidden_vecs'])[1]

    # [b, t, 1, h]
    return tf.tile(self.learned_query_vec, [b, t, 1])

  def call(self, input_dict):
    # [b, t, h]
    hidden_vecs = input_dict['hidden_vecs']

    # Expand the attention mask with new dims so that Keras can broadcast to
    # the same shape as the attn_score: [b, num_heads, a, t, a, t].
    # attn_mask shape: [b, 1, 1, 1 t,]
    # True means the position participate in the attention while all
    # False positions are ignored.
    
    #print("input batch shape: ", hidden_vecs.shape)
    
    has_historic_data = tf.logical_and(
          input_dict['has_historic_data'][..., 0],
          tf.logical_not(input_dict['is_hidden'][..., 0]))
    attn_mask = has_historic_data[:, tf.newaxis, tf.newaxis, :]
    # [b, t, 1, h]
    learned_query = self.build_learned_query(input_dict)
    attn_out, attn_score = self.attn_layer(
        query=learned_query,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=attn_mask,
        return_attention_scores=True)

    attn_out = self.attn_ln(attn_out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_ln(out + attn_out)


    return out, attn_score

In [8]:
class SelfAttnTransformerLayer(tf.keras.layers.Layer):
  """Performs full self-attention across the agent and time dimensions."""

  def __init__(
      self,
      num_heads=8,
      hidden_size=128,
      drop_prob=0.1,
      ln_eps=1e-6,
      ff_dim=128,
      mask=False,
      flatten=False,
      multimodality_induced=False,
  ):
    super().__init__()
    self.hidden_size = hidden_size
    self.mask = mask
    self.flatten = flatten
    self.multimodality_induced = multimodality_induced
    if hidden_size % num_heads != 0:
      raise ValueError(
          f'hidden_size ({hidden_size}) must be an integer '
          f'times bigger than num_heads ({num_heads}).'
      )
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=1,
    )  # Full Attention time
    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu'
    )
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None,
    )
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

  def call(self, input_dict, training=None):
    # [b, t, h] or [b, t, n, h]
    hidden_vecs = input_dict["hidden_vecs"]
    print("hidden_vecs", hidden_vecs.shape)

    if self.flatten:
      h_shape = tf.shape(hidden_vecs)
      b = h_shape[0]
      t = h_shape[1]
      h = h_shape[-1]

      if self.multimodality_induced:
        n = h_shape[2]
        hidden_vecs = tf.reshape(hidden_vecs, (b, -1, n, h))
      else:
        hidden_vecs = tf.reshape(hidden_vecs, (b, -1, h))

    # Expand the attention mask with new dims so that Keras can broadcast to
    # the same shape as the attn_score: [b, num_heads, a, t, a, t].
    # attn_mask shape: [b, 1, 1, 1, a, t,]
    # True means the position participate in the attention while all
    # False positions are ignored.
    if not self.mask:
      attn_mask = None
    else:
      print("has_historic_data", input_dict['has_historic_data'][..., 0].shape)
      has_historic_data = input_dict['has_historic_data'][..., 0]
      attn_mask = has_historic_data[:, tf.newaxis, tf.newaxis, :]
      print("attn_mask", attn_mask.shape)

    if attn_mask is not None and self.flatten:
      t = h_shape[1]

      if self.multimodality_induced:  # We have modes
        n = h_shape[2]
        attn_mask_with_modes = attn_mask[..., tf.newaxis, :, :]
        tiled_mask = tf.tile(attn_mask_with_modes, [1, 1, 1, 1, t])
        attn_mask = tf.reshape(
            tiled_mask,
            [b, 1, 1, tf.cast(t*n, tf.int32), tf.cast(t*n, tf.int32)]
        )
      else:
        tiled_mask = tf.tile(attn_mask, [1, 1, t, 1])
        attn_mask = tf.reshape(
            tiled_mask,
            [b, 1, tf.cast(t, tf.int32), tf.cast(t, tf.int32)]
        )


    attn_out, attn_score = self.attn_layer(
        query=hidden_vecs,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=attn_mask,
        return_attention_scores=True)
    out = self.attn_dropout(attn_out, training=training)
    attn_out = self.attn_ln(out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    if self.flatten:
      out = tf.reshape(out, h_shape)

    return out, attn_score

In [9]:
class SelfAttnModeTransformerLayer(tf.keras.layers.Layer):
  """Performs full self-attention across the future modes dimensions."""

  def __init__(self,
               num_heads=8,
               hidden_size=128,
               drop_prob=0.1,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.hidden_size = hidden_size
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=2)  # Attention over modes
    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

  def call(self, input_dict, training=None):

    # [b, t, n, h]
    hidden_vecs = input_dict["hidden_vecs"]
    print("hidden_vecs 3", hidden_vecs.shape)

    attn_out, attn_score = self.attn_layer(
        query=hidden_vecs,
        key=hidden_vecs,
        value=hidden_vecs,
        attention_mask=None,
        return_attention_scores=True)
    out = self.attn_dropout(attn_out, training=training)
    attn_out = self.attn_ln(out + hidden_vecs)

    # Feed-forward layers.
    out = self.ff_layer1(attn_out)
    out = self.ff_layer2(out)
    out = self.ff_dropout(out, training=training)
    out = self.ff_ln(out + attn_out)

    return out, attn_score

In [10]:
class MultimodalityInduction(tf.keras.layers.Layer):
  """Enables the model to forward and predict multi-mode predictions.

  1) Features are broadcasted to number of modes and summed with learned mode
      tensors.
  2) Mixture Weights are generated by cross-attention over all dimensions
      between learned mode tensors and hidden tensors.
  """

  def __init__(self,
               num_modes=5,
               num_heads=8,
               hidden_size=128,
               drop_prob=0.1,
               ln_eps=1e-6,
               ff_dim=128):
    super().__init__()
    self.num_modes = num_modes
    self.hidden_size = hidden_size
    if hidden_size % num_heads != 0:
      raise ValueError(f'hidden_size ({hidden_size}) must be an integer '
                       f'times bigger than num_heads ({num_heads}).')
    self.mm_attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=2)
    self.mm_attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.mm_ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.mm_ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=hidden_size,
        bias_axes='h',
        activation=None)
    self.mm_ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.mw_attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=hidden_size // num_heads,
        attention_axes=None)
    self.mw_attn_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)
    self.mw_ff_layer1 = tf.keras.layers.EinsumDense(
        '...h,hf->...f', output_shape=ff_dim, bias_axes='f', activation='relu')
    self.mw_ff_layer2 = tf.keras.layers.EinsumDense(
        '...f,fh->...h',
        output_shape=1,  # Single logit per mode
        bias_axes='h',
        activation=None)
    self.mw_ff_ln = tf.keras.layers.LayerNormalization(epsilon=ln_eps)

    self.attn_dropout = tf.keras.layers.Dropout(drop_prob)
    self.ff_dropout = tf.keras.layers.Dropout(drop_prob)

    # [1, 1, ,m,h]
    self.learned_add_mm = tf.Variable(
        tf.random_uniform_initializer(
            minval=-1.,
            maxval=1.)(shape=[1, 1, self.num_modes, hidden_size]),
        trainable=True,
        dtype=tf.float32)

  def call(self, input_dict, training=None):
    hidden_vecs = input_dict["hidden_vecs"]
    # [b, t, 1, h]
    hidden_vecs = hidden_vecs[..., tf.newaxis, :]

    # Multi Modes
    mm_add = self.mm_attn_ln(self.learned_add_mm + hidden_vecs)

    # Feed-forward layers.
    out = self.mm_ff_layer1(mm_add)
    out = self.mm_ff_layer2(out)
    out = self.ff_dropout(out)
    out = self.mm_ff_ln(out + hidden_vecs)

    # Mixture Weights
    # [b, 1, n, h]
    b = tf.shape(out)[0]
    attn_out_mw = self.mw_attn_layer(
        query=tf.tile(self.learned_add_mm, [b, 1, 1, 1]),
        key=mm_add,
        value=mm_add,
        return_attention_scores=False)
    attn_out_mw = self.attn_dropout(attn_out_mw, training=training)

    # [b, 1, n, h]
    attn_out_mw = self.mw_attn_ln(attn_out_mw)

    # Feed-forward layers.
    out_mw = self.mw_ff_layer1(attn_out_mw)
    out_mw = self.mw_ff_layer2(out_mw)
    out_mw = self.ff_dropout(out_mw, training=training)

    # [b, 1, n]
    mixture_logits = out_mw[..., 0]
    return out, mixture_logits

In [11]:
# Adapted from 2D Predictions to 3D predictions
class Prediction2DPositionHeadLayer(tf.keras.layers.Layer):
  """Converts transformer hidden vectors to model predictions."""

  def __init__(self, hidden_units=None, num_stages=5):
    super().__init__()

    self.dense_layers = []
    # Add hidden layers.
    if hidden_units is not None:
      for units in hidden_units:
        self.dense_layers.append(
            tf.keras.layers.Dense(units, activation='relu'))
    # Add the final prediction head.
    self.dense_layers.append(
        tf.keras.layers.EinsumDense(
            '...h,hf->...f',
            output_shape=9,
            bias_axes='f',
            activation=None))

  def call(self, input_dict):
    # [b, t, n,  h]
    hidden_vecs = input_dict["hidden_vecs"]

    x = hidden_vecs
    # [b, t, n, 5]
    for layer in self.dense_layers:
      x = layer(x)
    pred = x
    """predictions = {
        'agents/position': pred[..., 0:3],
        'agents/position/raw_scale_tril': pred[..., 3:5],
    }"""
    return pred

In [59]:
class HST(tf.keras.Model):
    def __init__(self, input_length):
        super().__init__()

        hidden_size = 128
        self.preprocess_layer = PreprocessLayer_2() 
        self.agent_pose_encoder = AgentKeypointsEncoder(output_shape=hidden_size-8, embedding_size=hidden_size)
        self.agent_pos_encoder = AgentPositionEncoder(output_shape=hidden_size-8, embedding_size=hidden_size)
        self.agent_temp_encoder = AgentTemporalEncoder(output_shape=hidden_size-8, embedding_size=hidden_size, num_steps=input_length)
        self.agent_encoder = FeatureConcatAgentEncoderLayer(input_length=input_length)
        self.align_layer = AgentSelfAlignmentLayer()
        self.tranformer1 = SelfAttnTransformerLayer(mask=True)
        self.multimodality_induction = MultimodalityInduction()
        self.tranformer3= SelfAttnTransformerLayer(mask=True, multimodality_induced=True)
        self.transformer4 = SelfAttnModeTransformerLayer()
        self.prediction_layer = Prediction2DPositionHeadLayer()
        

    def call(self, input_batch, training = False):
        (input_1, input_2) = input_batch
        output_dict = self.preprocess_layer((input_1, input_2)) # output shape (batch_size, 15, 3)
  
        encoded_keys, mask1 = self.agent_pose_encoder(output_dict)
        encoded_pos, mask2 = self.agent_pos_encoder(output_dict)
        encoded_temp, mask3 = self.agent_temp_encoder(output_dict)
        hidden_vecs = self.agent_encoder(output_dict)
        output_dict["hidden_vecs"] = hidden_vecs
        self_encoded_agent, _ = self.align_layer(output_dict)
        output_dict["hidden_vecs"] = self_encoded_agent
        hidden_vecs, _ = self.tranformer1(output_dict)
        hidden_vecs2, mixture_logits = self.multimodality_induction(output_dict)
        output_dict["hidden_vecs"] = hidden_vecs2
        hidden_vecs3, _ = self.tranformer3(output_dict)
        output_dict["hidden_vecs"] = hidden_vecs3
        hidden_vecs4, _ = self.transformer4(output_dict)
        output_dict["hidden_vecs"] = hidden_vecs4
        pred = self.prediction_layer(output_dict)


        output_dict={
            "encoded_keys": encoded_keys,
            "encoded_pos": encoded_pos,
            "encoded_temp": encoded_temp,
            "encoded_keys_mask": mask1,
            "encoded_pos_mask": mask2,
            "encoded_temp_mas": mask3,
            "output_dict": output_dict,
            "hidden_vecs": hidden_vecs,
            "hidden_vecs2": hidden_vecs2,
            "hidden_vecs3": hidden_vecs3,
            "mixture_logits": mixture_logits,
            'pred_pos': pred[...,0:3]
        }

        return output_dict

In [60]:
strategy = tf.distribute.OneDeviceStrategy('gpu')
with strategy.scope():
    model = HST(15)
    model.compile(loss='msle', optimizer='rmsprop')
for (batch_x1, batch_x2) in train_dataset.take(1):
    input_batch = (batch_x1, batch_x2)
    output = model(input_batch, training=False)
    break

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 16222427923489919739


is_hidden (1, 15, 1)
input_batch_new 0 (32, 15, 3)
input_batch_new 1 (32, 15, 51)
has_data (32, 15, 1)
has_historic_data (32, 1, 1)
should_predict (32, 15, 1)
masked_input_pos (32, 15, 3)
masked_input_pose (32, 15, 51)
(32, 15, 1, 120)
(32, 15, 120)
(32, 15, 1, 120)
(32, 15, 120)
(32, 15, 1, 120)
(32, 15, 120)
concat embedding (32, 15, 360)
hidden_vecs (32, 15, 128)
has_historic_data (32, 1)
attn_mask (32, 1, 1, 1)
hidden_vecs (32, 15, 5, 128)
has_historic_data (32, 1)
attn_mask (32, 1, 1, 1)
hidden_vecs 3 (32, 15, 5, 128)


In [124]:
output["pred_pos"]

<tf.Tensor: shape=(32, 15, 5, 3), dtype=float32, numpy=
array([[[[-0.58479565,  1.1494462 ,  1.1985414 ],
         [-0.26594967,  0.9777623 ,  1.1316311 ],
         [-0.48050267,  1.0369124 ,  1.1699543 ],
         [-0.5627724 ,  1.1177565 ,  1.1594312 ],
         [-0.41522986,  0.9024835 ,  1.1322446 ]],

        [[-0.91070414,  0.9054512 ,  1.0719429 ],
         [-0.5814908 ,  0.73537123,  1.012743  ],
         [-0.80036604,  0.7546586 ,  1.0327294 ],
         [-0.87831354,  0.85735786,  1.0260043 ],
         [-0.71396804,  0.64900315,  1.0046213 ]],

        [[-0.7963791 ,  0.45825484,  0.82039595],
         [-0.48476166,  0.29880816,  0.7310183 ],
         [-0.6578388 ,  0.36424407,  0.80825573],
         [-0.76236796,  0.39674795,  0.7418035 ],
         [-0.6185138 ,  0.19237648,  0.71325004]],

        ...,

        [[-0.01664364,  1.5041437 , -1.7927768 ],
         [ 0.4392018 , -0.29263675, -1.3876725 ],
         [ 0.7232453 ,  0.26380447, -2.2020106 ],
         [ 1.5644839 , -

In [108]:
output["encoded_agent"][0][3]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.81089574,  0.79600906, -0.70223266,  0.28319982, -1.0845588 ,
       -1.1686233 , -0.36819622,  0.2499477 , -0.6736832 ,  1.7577071 ,
        0.24864085, -0.01561074, -0.9247952 ,  0.5717657 , -0.05981993,
       -0.58544475, -0.6305727 ,  0.23966281, -0.04127312, -0.07515548,
       -0.7684814 ,  0.46016762, -1.2552994 , -0.11084849,  0.38545057,
       -0.35010007, -0.65156966,  0.43060076, -1.8044866 ,  0.42750764,
        1.9555049 , -0.39100063, -0.37983632,  0.31078747,  0.23831747,
       -0.8058383 , -0.569484  , -0.13494697,  0.38479537,  0.406515  ,
        0.21507372,  1.5170391 , -0.45104837,  0.02057071,  0.25320405,
        0.03062555,  0.24210678, -1.5947713 ,  0.8438032 , -0.6528839 ,
        0.36894557, -0.13338293, -0.70126474,  0.5603812 , -0.19411625,
        1.1276221 , -0.58010834,  0.73732305, -0.47832206,  0.19454718,
        1.0612475 , -0.13222125, -0.3588956 , -0.8896779 , -0.35958642,
        0.272765